In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
import re
from torch.utils.data import Dataset, DataLoader
import torch
import copy
from pysentimiento import create_analyzer

In [2]:
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

## Dataset

In [3]:
filenameDataset = "datasets/IMDBDatasetSPANISH.csv"
data = pd.read_csv(filenameDataset)
data.head()

,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


In [4]:
data.isnull().sum()

Unnamed: 0     0
review_en      0
review_es      0
sentiment      0
sentimiento    0
dtype: int64

In [5]:
def to_sentiment(sentiment):    
    sentiment = str(sentiment)    
    if sentiment == 'positive':
        return 1
    elif sentiment == 'negative':
        return 0
    else:
        return 2

In [6]:
#Check spanish
def selectLanguage(language, data):
    if language == "spanish":
        data['is_positive'] = data.sentiment.apply(to_sentiment)
        del data['review_en']
        del data['sentimiento']
        del data['sentiment']
        data.columns = ['id_review', 'text', 'sentiment']
    elif language == "english":
        del data['review_es']
        del data['sentimiento']
        del data[data.columns[0]]
        data.rename(columns={'review_en': 'text'}, inplace=True)
        data['sentiment'] = data.sentiment.apply(to_sentiment)
    return data

In [7]:
def basic_cleaning(text):
    text=re.sub(r'https?://www\.\S+\.com','',text)
    text=re.sub(r'[^A-Za-z|\s]','',text)
    text=re.sub(r'\*+','swear',text)
    return text

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F" #emoticons
                               u"\U0001F300-\U0001F5FF" #symbols & pictographs
                               u"\U0001F680-\U0001F6FF" #transport & map symbols
                               u"\U0001F1E0-\U0001F1FF" #flags (iOS)
                               u"\U00002702-\U000027B0" 
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_multiplechars(text):
    text = re.sub(r'(.)\1{3,}',r'\1', text)
    return text

def clean(df):
    for col in ['text']:#,'selected_text']:
        df[col]=df[col].astype(str).apply(lambda x:basic_cleaning(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_emoji(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_html(x))
        df[col]=df[col].astype(str).apply(lambda x:remove_multiplechars(x))
    return df

In [8]:
idioma = "spanish"
data = selectLanguage(idioma, data)

In [9]:
data.head()

,id_review,text,sentiment
0,0,Uno de los otros críticos ha mencionado que de...,1
1,1,Una pequeña pequeña producción.La técnica de f...,1
2,2,Pensé que esta era una manera maravillosa de p...,1
3,3,"Básicamente, hay una familia donde un niño peq...",0
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",1


In [10]:
print(data.sentiment.value_counts())
print(len(data.sentiment))

1    25000
0    25000
Name: sentiment, dtype: int64
50000


In [11]:
commentDFTrain, commentDFVal = train_test_split(data, test_size=0.2, random_state=seed) 

In [12]:
commentDFVal

,id_review,text,sentiment
26247,26247,"Sin héroes muertos, se obtienen líneas estúpid...",0
35067,35067,Pensé que tal vez ... tal vez esto podría ser ...,0
34590,34590,"Un equipo militar de élite americano que, por ...",0
16668,16668,Ridículo cine de terror sobre un hombre rico (...,0
12196,12196,"Bueno, si eres una de esas tuercas de cine de ...",1
...,...,...,...
49858,49858,Me gustó esta película.Eso es casi todo lo que...,1
19849,19849,"Imagina la difícil situación de Richard, un pi...",1
46899,46899,Hay un problema inherente al comentar o revisa...,0
28256,28256,Cuando la matriz apareció en 1999 y cuestionó ...,1


In [16]:
commentDFVal_clean = clean(commentDFVal)
commentDFVal_clean

,id_review,text,sentiment
26247,26247,Sin hroes muertos se obtienen lneas estpidas c...,0
35067,35067,Pens que tal vez tal vez esto podra ser bueno...,0
34590,34590,Un equipo militar de lite americano que por su...,0
16668,16668,Ridculo cine de terror sobre un hombre rico Jo...,0
12196,12196,Bueno si eres una de esas tuercas de cine de K...,1
...,...,...,...
49858,49858,Me gust esta pelculaEso es casi todo lo que pu...,1
19849,19849,Imagina la difcil situacin de Richard un pinto...,1
46899,46899,Hay un problema inherente al comentar o revisa...,0
28256,28256,Cuando la matriz apareci en y cuestion la exi...,1


In [17]:
def selectTexts(dataSet, words):
    list_text= []
    list_label= []
    for (text, target) in zip(dataSet.text, dataSet.sentiment):
        if (len(text.split())) <= words:
            list_text.append(text)
            list_label.append(target)
    DFTrain_clean = list(zip(list_text, list_label))
    DFTrain_clean = pd.DataFrame(DFTrain_clean, columns=['text','sentiment'])
    return DFTrain_clean 

In [18]:
commentDFVal_clean = selectTexts(commentDFVal_clean, 80)

In [19]:
commentDFVal_clean

,text,sentiment
0,Quieres saber lo que los escritores de esta pe...,0
1,Esta fue una pelcula muy bien guionadaGran div...,1
2,La pelcula no fue tan grandeEl libro es mejorP...,1
3,He tenido que cambiar mi opinin sobre la peor ...,0
4,El mundo entero est cayendo presa de una enfer...,1
...,...,...
887,Quin escribi el script para esta pelcula el pe...,0
888,Esta es la segunda pelcula que he visto de Ida...,1
889,Esta pelcula chupEl primero fue mucho mejorNad...,0
890,Me encant esta pelcula lo vi cuando tena unos ...,1


## Resultados

In [20]:
analyzer = create_analyzer(task="sentiment", lang="es")

In [21]:
def get_predictions_pysentimiento(dataset):
    r = []
    for i in dataset.text:
        sentiment = analyzer.predict(i).output
        sentiment = str(sentiment)    
        if sentiment == 'POS':
            number_sentiment = 1
        elif sentiment == 'NEG':
            number_sentiment = 0
        else:
            number_sentiment = 2

        r.append(number_sentiment)
    return r

In [22]:
def respuestas(reales, predicciones):
    contador_aciertos = 0
    contador_no_aciertos = 0
    for (real, prediccion) in zip(reales, predicciones):
        if real == prediccion:
            contador_aciertos = contador_aciertos+1
        else:
            contador_no_aciertos = contador_no_aciertos + 1
    return contador_aciertos, contador_no_aciertos  

In [29]:
def remove_value_two(df):
    df = df.drop(df[df['pysentimiento'] == 2].index)
    return df


In [23]:
label_preds = get_predictions_pysentimiento(commentDFVal_clean)

/Users/jose/miniforge3/envs/myclone/lib/python3.10/site-packages/transformers/models/roberta/modeling_roberta.py:1561: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:406.)
  incremental_indices = (torch.cumsum(mask, dim=1).type_as(mask) + past_key_values_length) * mask


In [24]:
label_preds

[0,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 2,
 0,
 2,
 0,
 1,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,


In [25]:
commentDFVal_clean['pysentimiento'] = label_preds

In [32]:
commentDFVal_clean.head()

,text,sentiment,pysentimiento
0,Quieres saber lo que los escritores de esta pe...,0,0
1,Esta fue una pelcula muy bien guionadaGran div...,1,1
2,La pelcula no fue tan grandeEl libro es mejorP...,1,1
3,He tenido que cambiar mi opinin sobre la peor ...,0,0
4,El mundo entero est cayendo presa de una enfer...,1,2


In [55]:
commentDFVal_clean

,text,sentiment,pysentimiento
0,Quieres saber lo que los escritores de esta pe...,0,0
1,Esta fue una pelcula muy bien guionadaGran div...,1,1
2,La pelcula no fue tan grandeEl libro es mejorP...,1,1
3,He tenido que cambiar mi opinin sobre la peor ...,0,0
4,El mundo entero est cayendo presa de una enfer...,1,2
...,...,...,...
887,Quin escribi el script para esta pelcula el pe...,0,2
888,Esta es la segunda pelcula que he visto de Ida...,1,0
889,Esta pelcula chupEl primero fue mucho mejorNad...,0,0
890,Me encant esta pelcula lo vi cuando tena unos ...,1,2


In [56]:
filePY = "output_pysentimiento"
commentDFVal_clean.to_csv(filePY, sep=',', index=False, encoding='utf-8')

In [30]:
DFoutput = remove_value_two(commentDFVal_clean)
DFoutput.head()

,text,sentiment,pysentimiento
0,Quieres saber lo que los escritores de esta pe...,0,0
1,Esta fue una pelcula muy bien guionadaGran div...,1,1
2,La pelcula no fue tan grandeEl libro es mejorP...,1,1
3,He tenido que cambiar mi opinin sobre la peor ...,0,0
5,Prncipe de Central Park es tan absolutamente ...,0,0


In [36]:
DFoutput

,text,sentiment,pysentimiento
0,Quieres saber lo que los escritores de esta pe...,0,0
1,Esta fue una pelcula muy bien guionadaGran div...,1,1
2,La pelcula no fue tan grandeEl libro es mejorP...,1,1
3,He tenido que cambiar mi opinin sobre la peor ...,0,0
5,Prncipe de Central Park es tan absolutamente ...,0,0
...,...,...,...
885,Esta es una de las mejores pelculas romnticas ...,1,1
886,Por qu esta pelcula fallaron comercialmenteTie...,1,1
888,Esta es la segunda pelcula que he visto de Ida...,1,0
889,Esta pelcula chupEl primero fue mucho mejorNad...,0,0


In [37]:

aciertos, no_aciertos = respuestas(DFoutput.sentiment, DFoutput.pysentimiento)

In [38]:
aciertos

694

In [39]:
no_aciertos

77

In [40]:
fileBERT = ""
DFoutput.to_csv(fileBERT, sep=',', index=False, encoding='utf-8')